In [ ]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only

True

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df=pd.read_json('https://raw.githubusercontent.com/LasseRegin/medical-question-answer-data/master/questionDoctorQAs.json')
#df=df.apply(lambda x: x.str.lower())
df[60:70]

,answer,answer_author,question,question_text,tags,url
60,bleeding after cold coagulation can occur for ...,Dr.Honey,i had cold coagulation for (cin) cervical intr...,I had cold coagulation for (cin) cervical intr...,"[cervical intraepithelial neoplasia, cold coag...",https://questiondoctors.com/i-had-cold-coagula...
61,it’s a normal sequelle after the coagulation a...,Dr Masr,i had cold coagulation for (cin) cervical intr...,I had cold coagulation for (cin) cervical intr...,"[cervical intraepithelial neoplasia, cold coag...",https://questiondoctors.com/i-had-cold-coagula...
62,ecg brain mri and echo. should be done asapwha...,Dr Masr,my husband is 42y/o hx. of stroke 1 year ago. ...,My husband is 42y/o Hx. of stroke 1 year ago. ...,[stroke],https://questiondoctors.com/my-husband-is-42yo...
63,hi this is not common set back go to the er an...,Dr Ahmed Fawzy,my husband is 42y/o hx. of stroke 1 year ago. ...,My husband is 42y/o Hx. of stroke 1 year ago. ...,[stroke],https://questiondoctors.com/my-husband-is-42yo...
64,that’s an emergency it can be a tia warning so...,Dr. Ayman Darrag,my husband is 42y/o hx. of stroke 1 year ago. ...,My husband is 42y/o Hx. of stroke 1 year ago. ...,[stroke],https://questiondoctors.com/my-husband-is-42yo...
65,the symptoms looks like a medical emergency. c...,Dr.Honey,my husband is 42y/o hx. of stroke 1 year ago. ...,My husband is 42y/o Hx. of stroke 1 year ago. ...,[stroke],https://questiondoctors.com/my-husband-is-42yo...
66,you will need to go to er for urgent tests.,Dr. D. Zluf,my husband is 42y/o hx. of stroke 1 year ago. ...,My husband is 42y/o Hx. of stroke 1 year ago. ...,[stroke],https://questiondoctors.com/my-husband-is-42yo...
67,hi it sounds like you have psos i need to know...,Dr Ahmed Fawzy,i have irregular periods and i am trying to ge...,I have Irregular periods and I am trying to ge...,[irregular periods pregnancy],https://questiondoctors.com/i-have-irregular-p...
68,hi unfortunately your only option would be sur...,Dr Ahmed Fawzy,i broke my collarbone back in 2009 my chest se...,"I broke my collarbone back in 2009, my chest s...",[spine deformation],https://questiondoctors.com/i-broke-my-collarb...
69,i need more details about your history your ag...,Dr. Ayman Darrag,i have irregular periods and i am trying to ge...,I have Irregular periods and I am trying to ge...,[irregular periods pregnancy],https://questiondoctors.com/i-have-irregular-p...


In [ ]:
df['question_text'].unique().shape

(1890,)

In [ ]:
data=pd.read_json('https://raw.githubusercontent.com/LasseRegin/medical-question-answer-data/master/webmdQAs.json')

In [ ]:

questions=[]
for que in df['question_text']:
  questions.append(que)
  




'''
sent_tokens='.'.join(sent_tokens)

print(sent_tokens)
sent_tokens = nltk.sent_tokenize(sent_tokens) # converts to list of sentences 
print(sent_tokens)
word_tokens = nltk.word_tokenize(sent_tokens) # converts to list of words
word_tokens'''

"\nsent_tokens='.'.join(sent_tokens)\n\nprint(sent_tokens)\nsent_tokens = nltk.sent_tokenize(sent_tokens) # converts to list of sentences \nprint(sent_tokens)\nword_tokens = nltk.word_tokenize(sent_tokens) # converts to list of words\nword_tokens"

# TF -IDF

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(word_tokens):
    return [lemmer.lemmatize(token) for token in word_tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [ ]:
GREETING_INPUTS = ('Issues', 'in', 'breathing', '.', 'Pain', 'in', 'chest', 'and', 'blood', 'clotted', 'cough')  #keywords from database
related_que=[]
respective_ans=[]
def greeting(sentence):
  for word in sentence.split():
    for response in df['question_text'][:30]:     #For each ques
       ques=nltk.word_tokenize(response)
       #print(ques)
       related_que.append(random.choice(ques))    #get randomized output 
       respective_ans.append(df['answer'])



In [ ]:
def fetch_features(chat):
    features = {}
    for word in nltk.word_tokenize(chat):
        features['contains({})'.format(word.lower())] = True
    return features

In [ ]:
answers=[]
def response(sent):
    robo_response=''
    questions.append(sent)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(questions)        #TF IDF Vector transformation (Word embedding, Count Vector, BOW, Glove alternates)
    vals = cosine_similarity(tfidf[-1], tfidf)       #Cosine similarity on TF IDF vectors 

    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        for i in range(10):
          answers.append(questions[idx-i])
        robo_response = robo_response+questions[idx]
        return robo_response


In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
user_response = 'chest pain in lungs'
user_response=user_response.lower()
if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you' ):
        flag=False
        print("ROBO: You are welcome..")
    else:
        print("ROBO: ",end="")
        print(response(user_response))
        questions.remove(user_response)
else:
    flag=False
    print("ROBO: Bye! take care..")
    print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
ROBO: I am (suffering) pain on left side of my chest?


In [ ]:
answers

['I am (suffering) pain on left side of my chest?',
 'I have been suffering from prostatitis, there is presence of pseudomonas fluorescens, need second opinion?',
 'I have been suffering from prostatitis, there is presence of pseudomonas fluorescens, need second opinion?',
 'I have been suffering from prostatitis, there is presence of pseudomonas fluorescens, need second opinion?',
 'I have been suffering from prostatitis, there is presence of pseudomonas fluorescens, need second opinion?',
 'I may have henoch schonlein purpura, need confirmation and advice?',
 'I may have henoch schonlein purpura, need confirmation and advice?',
 'I may have henoch schonlein purpura, need confirmation and advice?',
 'I may have henoch schonlein purpura, need confirmation and advice?',
 'There is bright red blood from my anus when I have bowel movement and when urinating?']